Primero se cargan las librerías que se van a usar

In [47]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import whois
import builtwith
import argparse
import requests
import re

Ahora se define la función para obtener el robots.txt

In [48]:
def obt_robot(siteurl):
    if siteurl.endswith('/'):
        path=siteurl
    else:
        path=siteurl+'/'
    req=requests.get(path+'robots.txt',data=None)
    return req.text

Ahora vamos a guardar el sitio web de amazon para posteriormente obtener los datos del robtos.txt

In [49]:
web = obt_robot('https://www.amazon.com/')

Obtenemos ahora el robots.txt e imprimimos el resultado

In [50]:
data_robots = re.sub( r'<[^>]*>', ' ', web).strip()
print(data_robots)

User-agent: *
Disallow: /exec/obidos/account-access-login
Disallow: /exec/obidos/change-style
Disallow: /exec/obidos/flex-sign-in
Disallow: /exec/obidos/handle-buy-box
Disallow: /exec/obidos/tg/cm/member/
Disallow: /gp/aw/help/id=sss
Disallow: /gp/cart
Disallow: /gp/flex
Disallow: /gp/product/e-mail-friend
Disallow: /gp/product/product-availability
Disallow: /gp/product/rate-this-item
Disallow: /gp/sign-in
Disallow: /gp/reader
Disallow: /gp/sitbv3/reader
Disallow: /gp/richpub/syltguides/create
Disallow: /gp/gfix
Disallow: /gp/associations/wizard.html
Disallow: /gp/dmusic/order
Disallow: /gp/legacy-handle-buy-box.html
Disallow: /gp/aws/ssop
Disallow: /gp/yourstore
Disallow: /gp/gift-central/organizer/add-wishlist
Disallow: /gp/vote
Disallow: /gp/voting/
Disallow: /gp/music/wma-pop-up
Disallow: /gp/customer-images
Disallow: /gp/richpub/listmania/createpipeline
Disallow: /gp/content-form
Disallow: /gp/pdp/invitation/invite
Disallow: /gp/customer-reviews/common/du
Disallow: /gp/customer-re

Como se aprecia en el archivo robots.txt, este nos muestra los apartados dentro del sito web para los cuales no deberíamos acceder con nuestro proceso de extracción de datos y que se respetarán a lo largo del proyecto

Ahora se va a identificar las tecnologías presentes detras del sitio web de Amazon

In [51]:
tech_used = builtwith.parse('https://www.amazon.com/')
print(tech_used)

{'javascript-frameworks': ['Prototype', 'jQuery']}


Luego se van a buscar mas datos sobre el sitio web

In [52]:
w = whois.whois('amazon.com/')
print(w)

{
  "domain_name": [
    "AMAZON.COM",
    "amazon.com"
  ],
  "registrar": "MarkMonitor, Inc.",
  "whois_server": "whois.markmonitor.com",
  "referral_url": null,
  "updated_date": [
    "2019-05-07 20:09:37",
    "2019-08-26 12:19:56"
  ],
  "creation_date": [
    "1994-11-01 05:00:00",
    "1994-10-31 21:00:00"
  ],
  "expiration_date": [
    "2024-10-31 04:00:00",
    "2024-10-30 00:00:00"
  ],
  "name_servers": [
    "NS1.P31.DYNECT.NET",
    "NS2.P31.DYNECT.NET",
    "NS3.P31.DYNECT.NET",
    "NS4.P31.DYNECT.NET",
    "PDNS1.ULTRADNS.NET",
    "PDNS6.ULTRADNS.CO.UK",
    "pdns6.ultradns.co.uk",
    "ns3.p31.dynect.net",
    "ns4.p31.dynect.net",
    "ns2.p31.dynect.net",
    "ns1.p31.dynect.net",
    "pdns1.ultradns.net"
  ],
  "status": [
    "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited",
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited",
    "serverDele

Para realizar la busqueda de datos, primero usamos la librería HTMLSessions para transformar la estructuta de los datos y creamos una lista vacía para que posteriomente se puedan agregar las variables capturadas 

In [58]:
lista_de_descuentos = []
x = AsyncHTMLSession()

Ahora se va a empezar a trabajar en función de realizar la extracción de datos del sitio web. Para este proyecto vamos a realizar las busqueda de promociones para una marca de computadores, sin embargo,  si es necesario se puede cambiar el item de busqueda.

In [59]:
buscar = 'asus+zenbook'
url = f'https://www.amazon.com/s?k={buscar}'

Con la primera función que se va a crear se podrán acceder a la data del sitio web. Luego usamos sleep para evitar el bloqueo del servidor y posteriormente se le hace parseo al sitio por medio de la librería BeatifulSoup

In [60]:
def datos(url):
    y = x.get(url)
    y.html.render(sleep=1)
    soup = BeautifulSoup(y.html.html, 'html.parser')
    return soup

El siguiente paso consiste en crear la función que nos arroje la lista de los datos especificos que queremos del producto(título, título abreviado, link, precio actual en el caso si es que tiene descuento, precio sin descuento, críticas, calificación, si es un producto promocionado por amazon a través de amazon choice y la fecha de extracción de los datos)

In [61]:
def descuentos(soup):
# Se crea productos para almacenar allí la data temporal para el bucle
    productos = soup.find_all('div', {'data-component-type':'s-search-result'})
    amazonChoice=False
    for item in productos:
        # Busqueda del nombre completo del producto
        titulo = item.find('a', {'class': 'a-link-normal a-text-normal'}).text.replace(',',' ').strip()
                # Ahora buscamos la etiqueta de amazon choice (Amazon's) para marcar los articulos que la tengan. 
        try:
            if(item.find('span', {'class': 'a-badge-text', 'data-a-badge-color':'sx-cloud'}).text.replace(',',' ').strip() == "Amazon's"):
                 amazonChoice=True
            else:
                amazonChoice=False
        except:
            amazonChoise= False
        # Abreviación del título para mejorar comprensión
        titulo_abreviado = item.find('a', {'class': 'a-link-normal a-text-normal'}).text.replace(',',' ').strip()[:20]
        # Busqueda del link del producto
        link = item.find('a', {'class': 'a-link-normal a-text-normal'})['href']
        # Se crea primero una lista llamada lista_spam para verificar si el producto tiene o no descuento
        lista_span = item.find_all('span', {'class': 'a-offscreen'})
        precio_actual, precio_sin_descuento = 0, 0
        if not lista_span:
            print(titulo, "No matches")
        else:
            try:
                precio_actual = float(lista_span[0].text.replace('$','').replace(',','').strip())
                precio_sin_descuento = float(lista_span[1].text.replace('$','').replace(',','').strip())
            except:
                precio_sin_descuento = float(lista_span[0].text.replace('$','').replace(',','').strip())
        # Busqueda de Críticas del producto
        try:
            #criticas = item.find('span', {'class': "a-size-base"}, partial=False).text.strip()
            criticas = float(item.find(lambda tag: tag.name == 'span' and tag.get('class') == ['a-size-base']).text.strip())
        except:
            criticas = 0
        # Busqueda de la calificación en estrellas del producto
        try:
            calificacion = float(item.find('span', {'class': 'a-declarative'}).text.replace(' out of 5 stars','').strip())
        except:
            calificacion = 0
        # Fecha en la cuál fue extraída la información
        date = datetime.datetime.now()
        # Creación de la lista con las varibales obtenidas para cada producto
        articulo_en_venta= {
            'titulo': titulo,
            'titulo_abreviado': titulo_abreviado,
            'link': link,
            'precio_actual': precio_actual,
            'precio_sin_descuento': precio_sin_descuento,
            'criticas': criticas,
            'calificacion': calificacion,  
            'AmazonChoice': amazonChoice,
            'Date':  str(date.day)+'/'+str(date.month)+'/'+str(date.year)    
            }
        lista_de_descuentos.append(articulo_en_venta)
    return


Posteriormente se crea la función para realizar el salto de página dentro del sitio web de amazon

In [62]:
def siguiente_pagina(soup): 
    # Se realiza la busqueda de las páginas que contienen el producto
    pages = soup.find('ul', {'class': 'a-pagination'})
    # Condicional para continuar la busqueda hasta que encuentre la página final
    if not pages.find('li', {'class': 'a-disabled a-last'}):
        url = 'https://www.amazon.com' + str(pages.find('li', {'class': 'a-last'}).find('a')['href'])
        return url
    else:
        return
i=0
while i<20:
    soup = datos(url)
    descuentos(soup)
    url = siguiente_pagina(soup)
    i=i+1
    if not url:
        break
    else:
        print(url)

AttributeError: '_asyncio.Future' object has no attribute 'html'

Creamos una funcón para generar el dataframe

In [ ]:
i=0
while i<20:
    soup = datos(url)
    descuentos(soup)
    url = siguiente_pagina(soup)
    i=i+1
    if not url:
        break
    else:
        print(url)

In [ ]:
df = pd.DataFrame(lista_de_descuentos)
print(lista_de_descuentos)

Se calcula el porcentaje de descuento del producto y se ordena de mayor a menor

In [ ]:
#df['porcentaje_de_descuento'] = 100 - ((df.precio_actual / df.precio_sin_descuento) * 100)
df['porcentaje_de_descuento'] = 0
df = df.sort_values(by=['porcentaje_de_descuento'], ascending=False)

Se crea el archivo csv a partir de la información obtenida y se emite un mensaje de finalización de la ejecución del programa

In [ ]:
df.to_csv(buscar + 'descuentos_encontrados_giovanny333.csv', index=False)
print('La busqueda de ' + buscar + ' ha finalizado')